In [1]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/Users/fllorente/Dropbox/con_Petar/PYTHON/gp_fusion')


In [78]:
from modules.model_training import train_and_predict_single_gp
from modules.fusion_methods import compute_neg_log_like
from modules.model_training import GPModel, to_torch, train_variational_gp, predict_variational_gp

import torch
from tqdm import tqdm

from gpytorch.means import ZeroMean
from gpytorch.kernels import AdditiveStructureKernel, RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.metrics import mean_standardized_log_loss

In [12]:
import imageio.v2 as imageio
import imageio as iio
 
X_train = []
# read an image 
for i in 1+np.arange(12000):
    img = iio.imread(f"/Users/fllorente/Downloads/datasets_without_preprocessing/face_images/face_images_train/{i}.jpg")
    X_train.append(img)

X_train = np.array(X_train)    
X_train = X_train.reshape(12000,28*28)
X_train.shape

/var/folders/b6/z4591_kn5dz7xy81wtgqr67c0000gp/T/ipykernel_44864/4241675017.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(f"/Users/fllorente/Downloads/datasets_without_preprocessing/face_images/face_images_train/{i}.jpg")


(12000, 784)

In [13]:
X_test = []
# read an image 
for i in 1+np.arange(800):
    img = iio.imread(f"/Users/fllorente/Downloads/datasets_without_preprocessing/face_images/face_images_test/{i}.jpg")
    X_test.append(img)

X_test = np.array(X_test)    
X_test = X_test.reshape(800,28*28)
X_test.shape

/var/folders/b6/z4591_kn5dz7xy81wtgqr67c0000gp/T/ipykernel_44864/236637803.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(f"/Users/fllorente/Downloads/datasets_without_preprocessing/face_images/face_images_test/{i}.jpg")


(800, 784)

In [ ]:
# Normalize data
X_train = X_train/255.0
X_test = X_test/255.0

In [68]:
import pandas as pd

file = "/Users/fllorente/Downloads/datasets_without_preprocessing/face_images/face_images_train/train_list.txt"
df = pd.read_csv(file, header=None, delimiter=' ') # or quotechar = "'"
y_train = np.array(df[1])

file = "/Users/fllorente/Downloads/datasets_without_preprocessing/face_images/face_images_test/test_list.txt"
df = pd.read_csv(file, header=None, delimiter=' ') # or quotechar = "'"
y_test = np.array(df[1])

In [73]:
y_train.shape, y_test.shape
meanY = np.mean(y_train)
stdY = np.std(y_train)

In [74]:
# Normalize data
y_train = (y_train - meanY)/stdY
y_test = (y_test - meanY)/stdY

In [75]:
print(X_train.max(),X_test.max())
print(y_train.mean(), y_test.mean())
print(y_train.std(), y_test.std())

1.0 1.0
-7.105427357601002e-18 -0.011693234372475953
0.9999999999999999 1.0026059874361415


In [92]:
# Fit a GP with ARD-RBF kernel as the baseline
inducing_points = X_train[:5000]
model,likelihood = train_variational_gp(X_train, y_train, inducing_points, 
                                        learning_rate = 0.1, batch_size=128, num_epochs=10, 
                                        kappa=10, lambdaa = 10)

mean_tr,std_tr = predict_variational_gp(model, likelihood, X_train)
mean_tst,std_tst = predict_variational_gp(model, likelihood, X_test)


print("NLPD train: ", compute_neg_log_like(mean_tr,std_tr,y_train))
print("NLPD test: ", compute_neg_log_like(mean_tst,std_tst,y_test))

print("rmse train: ", np.sqrt(np.mean((y_train - mean_tr)**2)))
print("rmse test: ", np.sqrt(np.mean((y_test - mean_tst)**2)))

In [90]:
mean_tst[:5]

array([-0.02671797, -0.02671797, -0.02671797, -0.02671797, -0.02671797],
      dtype=float32)